In [1]:
#General imports needed for path
import os 
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping


import warnings
warnings.filterwarnings('ignore')

In [3]:
#Packages created by our group:
import route_dynamics.route_elevation.base_df as base
import route_dynamics.route_energy.knn as knn
from route_dynamics.route_riders import route_riders as ride
import route_dynamics.route_energy.longi_dynam_model as ldm
import route_dynamics.route_energy.accel as ca

In [4]:
#User defines what routes they want to evaluate
rt_list = [22,101,102,143,150,153,154,156,157,158,
           159,168,169,177,178,179,180,181,182,183,
           186,187,190,192]

#route = rt_list[0]

#time = 'AM' #AM, MID, PM, XEV, XNT
time_list_typical = ['MID', 'XNT', 'PM', 'AM', 'MID', 'MID', 'PM', 'XEV', 'PM', 'PM',
                     'PM', 'AM', 'MID', 'XEV', 'AM', 'AM', 'MID', 'AM', 'MID', 'MID',
                     'AM', 'MID', 'PM', 'PM']
time_list_easy = ['AM', 'XEV', 'XEV', 'XEV', 'AM', 'XEV', 'MID', 'AM', 'AM', 'XNT',
                     'XEV', 'XNT', 'XNT', 'MID', 'XEV', 'MID', 'XNT', 'XEV', 'XEV', 'XNT',
                     'XNT', 'XNT', 'AM', 'PM']
time_list_hard = ['PM', 'PM', 'AM', 'PM', 'XEV', 'AM', 'AM', 'PM', 'PM', 'AM',
                     'AM', 'PM', 'XEV', 'XNT', 'PM', 'PM', 'PM', 'PM', 'PM', 'PM',
                     'PM', 'PM', 'PM', 'AM']
#Import route shapefile

#Import bus stops shapefile

stops_shp = '../data/Transit_Stops_for_King_County_Metro__transitstop_point.shp'

signals_shp = '../data/traffic_signals2.shp'

trip183 = pd.read_csv("../data/Trip183.csv", usecols = ['SignRt', 'InOut', 'KeyTrip', 'BusType', 'Seats', 
                     'Period', 'AnnRides']) # KCM Data
trip183unsum = pd.read_csv("../data/Zon183Unsum.csv", usecols = ['Route', 'Dir', 'Trip_ID', 'InOut', 'STOP_SEQ', 'STOP_ID',
                     'Period', 'AveOn', 'AveOff', 'AveLd', 'Obs'])

#Acceleration profile
a_prof = pd.read_csv("../data/acceleration.csv", names=['time (s)', 'accel. (g)'])

In [ ]:
for i in range(4 , 5):
#for i in range(len(rt_list)):
    route = rt_list[i]
    shapefile_name = '../data/rt' + str(route) + '_pts2.shp'
    
    time = time_list_easy[i]
    trip_df = ride.route_ridership(time, route, 13041)
    geometry = trip_df.geometry.values
    avg_riders = trip_df['AveLd'].mean()

    xy = []
    for i in range(len(geometry)):
        dic = mapping(geometry[i][0])
        coords = dic['coordinates']
        xy.append(coords)
        xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
        xy_df['coordinates'] = xy

    stop_coords = xy_df.coordinates.values

    signals = gpd.read_file(signals_shp)
    signals = signals[signals['Route_Num']==str(route)]
    geometry = signals.geometry.values
    xy = []
    for i in range(len(geometry)):
        dic = mapping(geometry[i])
        coords = dic['coordinates']
        xy.append(coords)
        xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
        xy_df['coordinates'] = xy

    signal_coords = xy_df.coordinates.values

    mass_array = trip_df.Total_Mass.values

    ex = ldm.RouteTrajectory(
    route,
    shapefile_name,
    a_prof,
    stop_coords,
    signal_coords,
    mass_array = mass_array,
    unloaded_bus_mass = 13041,
    charging_power_max = 160000,
    aux = 7000
    )

    rt_df = ex.route_df
    #avg_speed_lim = rt_df['speed_limit'].mean()*3.6
    #total_time = rt_df['total_time'].iloc[-1]/60
    total_energy, total_time = ex.energy_from_route(rt_df)

    #print('Average speed limit for route ' + str(route) + ' is ' + str(avg_speed_lim))
    print('Total time ' + str(route) + ' is ' + str(total_time))
    print('Total energy for route ' + str(route) + ' is '+ str(total_energy))

    fig, ax = plt.subplots(figsize=(12, 5), dpi=300)

    ax.fill_between(rt_df['distance']/1000, rt_df['elevation'], color='#BDBDBD')
    ax1 = ax.twinx()

    ax1.plot(rt_df['distance']/1000, rt_df['power_output']/1000, linewidth=3)
    ax1.set_ylabel('Load (kW)', fontsize=20)
    ax1.set_ylim(-100, 250)
    #ax.set_ylim(0, 200)
    ax.set_xlabel('Distance (km)', fontsize=20)
    ax.set_ylabel('Elevation (m)', fontsize=20)
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    ax1.yaxis.set_label_position("left")
    ax1.yaxis.tick_left()

    ax.tick_params(labelsize=14)
    ax1.tick_params(labelsize=14)
    ax1.grid(axis='y')

    fig.suptitle(
        'Load Profile for Route {}'.format(route),
        fontsize=24,
        y=1,
        )
    plt.savefig('4_29/profile{}.png'.format(route), dpi=300)

    fig, ax = plt.subplots(figsize=(12, 5), dpi=300)

    bins = [-80, -60, -40, -20, 0, 20, 40, 60, 80, 100, 120, 140, 160,180,200]
    ax.hist(rt_df['power_output']/1000, bins=bins, weights=rt_df['delta_times']/60)
    ax.set_ylabel('Time(min)', fontsize=20)
    ax.set_xlabel('Load (kW)', fontsize=20)
    ax.tick_params(labelsize=12)
    ax.set_xticks(bins)

    fig.suptitle(
        'Histogram for Route {}'.format(route),
        fontsize=24,
        y=1,
        )

    plt.savefig('4_29/hist{}.png'.format(route), dpi=300)

    bins2 = [rt_df['power_output'].min()/1000,(rt_df['power_output'].min()/1000)+20, -40, -20, 0, 20, 40, 60, 80, 100, 120, 140, (rt_df['power_output'].max()/1000)-20, rt_df['power_output'].max()]
    results, bins = np.histogram(rt_df['power_output']/1000, bins=bins2, weights=rt_df['delta_times']/60)
    print('Total time at min bin for route ' + str(route) + ' is ' +str(results[0]))
    print('Total time at max bin for route ' + str(route) + ' is ' +str(results[-1]))
    
    rt_df['load_cell(W)'] = rt_df['power_output']/59260
    
    fig, ax = plt.subplots(1, 1, figsize = (10,6), dpi=100)

    ax.plot(total_time/60, rt_df['load_cell(W)'], linewidth=2)
    ax.set_xlabel('Time (min)', fontsize=20)
    ax.set_ylabel('Load (W)', fontsize=20)
    ax.tick_params(labelsize=14)
    ax.grid(axis='y')
    
    plt.savefig('4_29/cell{}.png'.format(route), dpi=300)

In [ ]:
#rt_df['total_force'] = rt_df.grav_force + rt_df.roll_fric + rt_df.aero_drag + rt_df.inertia              

In [ ]:
# fig, ax = plt.subplots(figsize=(12, 5), dpi=300)

# ax.plot(rt_df['distance']/1000, rt_df['total_force']/1000, linewidth=4, label = 'Total Force', color = 'black')
# ax.plot(rt_df['distance']/1000, rt_df['roll_fric']/1000, linewidth=3, label = 'Rolling Resistance', color = '#377eb8', linestyle='--')
# ax.plot(rt_df['distance']/1000, rt_df['aero_drag']/1000, linewidth=3, label = 'Air Drag Resistance', color = '#ff7f00', linestyle='--')
# ax.plot(rt_df['distance']/1000, rt_df['grav_force']/1000, linewidth=3, label = 'Gravitational Force', color = '#4daf4a', linestyle='--')
# ax.plot(rt_df['distance']/1000, rt_df['inertia']/1000, linewidth=3, label = 'Acceleration Force', color = '#f781bf', linestyle='--')

# ax.set_ylabel('Force (kN)', fontsize=20)
# #ax.set_xlim(5, 6)
# #ax.set_ylim(-30, 30)
# ax.set_xlabel('Distance (km)', fontsize=20)

# ax.tick_params(labelsize=14)

# ax.legend()

# fig.suptitle(
#     'Route {}'.format(route),
#     fontsize=24,
#     y=1,
#     )
# plt.savefig('4_29/force_profile{}.png'.format(route), dpi=300)